# 02 - Data Processing

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import numpy as np
import pandas as pd

import stock_utils.constants as cnst
import stock_utils.data as sd

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
SCREENER_EPS_DATA_DIR = PosixPath('../data/Screener.in/EPS') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
stock_symbols = sd.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['DEEPAKFERT',
 'HDFCBANK',
 'HDFCSML250',
 'INDIGOPNTS',
 'ITBEES',
 'JUBLFOOD',
 'NH']

In [4]:
STOCK_SYMBOL = stock_symbols[-1]
STOCK_SYMBOL

'NH'

## Data loading

In [5]:
stock_df = pd.read_parquet(cnst.NSE_DATA_DIR.joinpath(STOCK_SYMBOL, "consolidated.parquet"))
stock_df.columns = [
    'Date', 'Series',
    'Open', 'High', 'Low', 'Prev Close', 'LTP', 'Close',
    'VWAP', '52W H', '52W L',
    'Volume', 'Value', '#Trades'
] + (["PE"] if "PE" in stock_df.columns else [])
stock_df

,Date,Series,Open,High,Low,Prev Close,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades
0,2020-01-01,EQ,309.60,331.80,306.30,307.30,326.30,326.00,323.24,331.80,182.8,775112,2.505503e+08,23768
1,2020-01-02,EQ,329.00,333.00,321.30,326.00,331.50,329.75,327.19,333.00,182.8,226542,7.412127e+07,6804
2,2020-01-03,EQ,333.00,340.25,325.25,329.75,329.00,328.35,333.78,340.25,182.8,418178,1.395790e+08,18600
3,2020-01-06,EQ,325.00,342.00,320.05,328.35,339.00,337.65,331.16,342.00,182.8,253734,8.402544e+07,11824
4,2020-01-07,EQ,343.00,345.80,331.25,337.65,336.30,339.65,338.63,345.80,182.8,291647,9.875900e+07,8584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,2024-04-22,EQ,1244.30,1262.70,1242.50,1242.45,1250.40,1255.55,1254.02,1444.90,748.9,242623,3.042545e+08,21720
1070,2024-04-23,EQ,1255.55,1261.80,1242.20,1255.55,1245.25,1251.30,1250.41,1444.90,748.9,273737,3.422828e+08,30658
1071,2024-04-24,EQ,1261.95,1281.00,1256.40,1251.30,1264.55,1262.60,1268.95,1444.90,748.9,176087,2.234450e+08,24482
1072,2024-04-25,EQ,1273.10,1285.85,1257.05,1262.60,1260.30,1266.55,1271.82,1444.90,748.9,166628,2.119214e+08,28170


In [6]:
stock_df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1074 entries, 0 to 1073
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        1074 non-null   datetime64[ns]
 1   Series      1074 non-null   object        
 2   Open        1074 non-null   float64       
 3   High        1074 non-null   float64       
 4   Low         1074 non-null   float64       
 5   Prev Close  1074 non-null   float64       
 6   LTP         1074 non-null   float64       
 7   Close       1074 non-null   float64       
 8   VWAP        1074 non-null   float64       
 9   52W H       1074 non-null   float64       
 10  52W L       1074 non-null   float64       
 11  Volume      1074 non-null   int64         
 12  Value       1074 non-null   float64       
 13  #Trades     1074 non-null   int64         
dtypes: datetime64[ns](1), float64(10), int64(2), object(1)
memory usage: 171.1 KB


## Feature engineering

### Daily candle based columns

In [7]:
stock_df['Range'] = stock_df['High'] - stock_df['Low']
stock_df['IsGreen'] = (stock_df['Close'] >= stock_df['Prev Close']).astype(int)
stock_df['Is52WLow'] = np.isclose(stock_df['Low'], stock_df['52W L'], atol = 0, rtol = 1e-4).astype(int)
stock_df['Is52WHigh'] = np.isclose(stock_df['High'], stock_df['52W H'], atol = 0, rtol = 1e-4).astype(int)
stock_df = stock_df.drop(columns = ['Series', 'Prev Close'])
stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh
642,2022-07-28,628.20,660.00,621.65,657.0,658.75,648.37,777.00,468.30,210627,1.365633e+08,12096,38.35,1,0,0
463,2021-11-10,548.00,577.95,541.20,544.0,547.20,564.44,577.95,328.85,553338,3.123261e+08,24009,36.75,0,0,1
700,2022-10-24,730.00,734.05,726.35,730.0,730.95,730.62,777.00,501.80,18134,1.324907e+07,1389,7.70,1,0,0
748,2023-01-02,764.70,767.60,747.05,749.0,748.85,754.86,820.00,588.45,126976,9.584879e+07,7642,20.55,0,0,0
806,2023-03-27,745.55,752.30,723.10,745.0,744.30,744.26,821.60,588.45,107577,8.006538e+07,13841,29.20,0,0,0
214,2020-11-06,347.10,347.10,335.00,342.0,341.95,341.04,389.00,204.00,93789,3.198535e+07,6761,12.10,0,0,0
577,2022-04-27,707.95,716.50,700.10,710.0,710.05,704.83,777.00,393.05,398093,2.805871e+08,6145,16.40,1,0,0
519,2022-01-31,643.25,652.95,636.15,648.0,645.40,643.96,675.00,380.00,121151,7.801634e+07,10471,16.80,1,0,0
43,2020-03-02,340.45,344.70,325.00,326.6,329.95,338.92,389.00,184.30,220975,7.489283e+07,5276,19.70,0,0,0
765,2023-01-25,726.95,729.00,717.00,718.3,724.15,723.08,820.00,588.45,25378,1.835034e+07,3394,12.00,0,0,0


In [8]:
((stock_df[['IsGreen', 'Is52WLow', 'Is52WHigh']].sum() * 100) / len(stock_df)).round(2)

IsGreen      51.02
Is52WLow      0.00
Is52WHigh     6.24
dtype: float64

In [9]:
stock_df.shape

(1074, 16)

### Streak

In [10]:
stock_df["Streak"] = (stock_df["IsGreen"] != stock_df["IsGreen"].shift(1)).cumsum()
stock_df["Streak"] = stock_df.groupby("Streak").cumcount() + 1
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh,Streak
0,2020-01-01,309.60,331.80,306.30,326.30,326.00,323.24,331.80,182.8,775112,2.505503e+08,23768,25.50,1,0,1,1
1,2020-01-02,329.00,333.00,321.30,331.50,329.75,327.19,333.00,182.8,226542,7.412127e+07,6804,11.70,1,0,1,2
2,2020-01-03,333.00,340.25,325.25,329.00,328.35,333.78,340.25,182.8,418178,1.395790e+08,18600,15.00,0,0,1,1
3,2020-01-06,325.00,342.00,320.05,339.00,337.65,331.16,342.00,182.8,253734,8.402544e+07,11824,21.95,1,0,1,1
4,2020-01-07,343.00,345.80,331.25,336.30,339.65,338.63,345.80,182.8,291647,9.875900e+07,8584,14.55,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,2024-04-22,1244.30,1262.70,1242.50,1250.40,1255.55,1254.02,1444.90,748.9,242623,3.042545e+08,21720,20.20,1,0,0,2
1070,2024-04-23,1255.55,1261.80,1242.20,1245.25,1251.30,1250.41,1444.90,748.9,273737,3.422828e+08,30658,19.60,0,0,0,1
1071,2024-04-24,1261.95,1281.00,1256.40,1264.55,1262.60,1268.95,1444.90,748.9,176087,2.234450e+08,24482,24.60,1,0,0,1
1072,2024-04-25,1273.10,1285.85,1257.05,1260.30,1266.55,1271.82,1444.90,748.9,166628,2.119214e+08,28170,28.80,1,0,0,2


### Date based columns

In [11]:
stock_df['Day'] = stock_df['Date'].dt.day
stock_df['Month'] = stock_df['Date'].dt.month
stock_df['Year'] = stock_df['Date'].dt.year
stock_df['Weekday'] = stock_df['Date'].dt.weekday
stock_df['DayOfYear'] = stock_df['Date'].dt.day_of_year
stock_df['Quarter'] = stock_df['Date'].dt.quarter
stock_df['DaysSinceLastTradingSession'] = (stock_df['Date'] - stock_df['Date'].shift(1)).dt.days.fillna(1).astype(int)

stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession
617,2022-06-23,633.45,638.80,625.00,627.80,628.05,633.10,777.00,450.00,269916,1.708851e+08,17115,13.80,0,0,0,3,23,6,2022,3,174,2,1
844,2023-05-25,848.30,867.15,843.35,863.00,859.40,857.72,880.80,588.45,468530,4.018660e+08,27315,23.80,1,0,0,2,25,5,2023,3,145,2,1
948,2023-10-25,1032.05,1034.00,990.00,1009.00,1009.25,1010.44,1132.80,699.30,207714,2.098834e+08,18215,44.00,0,0,0,4,25,10,2023,2,298,4,2
202,2020-10-21,335.00,338.30,328.00,334.95,334.80,333.91,389.00,204.00,97008,3.239182e+07,6175,10.30,1,0,0,1,21,10,2020,2,295,4,1
1042,2024-03-07,1210.60,1216.85,1176.00,1183.10,1179.75,1190.86,1444.90,723.10,526411,6.268805e+08,46581,40.85,0,0,0,4,7,3,2024,3,67,1,1
926,2023-09-21,1079.80,1098.00,1054.60,1068.00,1070.45,1072.93,1121.00,683.35,339014,3.637370e+08,24772,43.40,0,0,0,3,21,9,2023,3,264,3,1
224,2020-11-20,355.00,392.00,355.00,369.80,367.65,374.26,392.00,204.00,754925,2.825378e+08,19392,37.00,1,0,1,1,20,11,2020,4,325,4,1
912,2023-08-31,1025.05,1046.40,1015.10,1041.50,1039.50,1038.22,1076.95,675.00,227183,2.358656e+08,17919,31.30,1,0,0,1,31,8,2023,3,243,3,1
832,2023-05-09,769.40,776.45,752.20,760.80,756.15,766.67,821.60,588.45,265862,2.038290e+08,17977,24.25,0,0,0,1,9,5,2023,1,129,2,1
474,2021-11-26,567.00,588.00,544.00,547.55,549.60,566.18,626.70,364.75,435784,2.467327e+08,29638,44.00,0,0,0,2,26,11,2021,4,330,4,1


In [12]:
stock_df.shape

(1074, 24)

### Moving averages

#### `Close`

In [13]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Close {window}MA'] = stock_df['Close'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Close.*")

,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA
0,326.00,326.00,326.00,326.00,326.00
1,329.75,327.88,327.88,327.88,327.88
2,328.35,328.03,328.03,328.03,328.03
3,337.65,330.44,330.44,330.44,330.44
4,339.65,332.28,332.28,332.28,332.28
...,...,...,...,...,...
1069,1255.55,1269.23,1284.41,1258.19,1299.32
1070,1251.30,1260.41,1282.24,1258.35,1299.66
1071,1262.60,1256.72,1280.00,1260.28,1300.03
1072,1266.55,1256.35,1278.27,1263.18,1300.11


#### `Range`

In [14]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Range {window}MA'] = stock_df['Range'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Range.*")

,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA
0,25.50,25.50,25.50,25.50,25.50
1,11.70,18.60,18.60,18.60,18.60
2,15.00,17.40,17.40,17.40,17.40
3,21.95,18.54,18.54,18.54,18.54
4,14.55,17.74,17.74,17.74,17.74
...,...,...,...,...,...
1069,20.20,40.08,40.42,45.32,46.36
1070,19.60,39.88,38.53,43.46,45.86
1071,24.60,35.44,37.51,42.17,45.67
1072,28.80,32.85,37.45,41.77,45.68


#### `VWAP`

In [15]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'VWAP {window}MA'] = stock_df['VWAP'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "VWAP.*")

,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA
0,323.24,323.24,323.24,323.24,323.24
1,327.19,325.22,325.22,325.22,325.22
2,333.78,328.07,328.07,328.07,328.07
3,331.16,328.84,328.84,328.84,328.84
4,338.63,330.80,330.80,330.80,330.80
...,...,...,...,...,...
1069,1254.02,1276.13,1289.47,1261.72,1301.57
1070,1250.41,1266.21,1288.10,1261.05,1301.75
1071,1268.95,1260.56,1286.62,1263.12,1302.21
1072,1271.82,1259.13,1284.77,1265.82,1302.44


#### `Volume`

In [16]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Volume {window}MA'] = stock_df['Volume'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Volume.*")

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,775112,775112,775112,775112,775112
1,226542,500827,500827,500827,500827
2,418178,473277,473277,473277,473277
3,253734,418391,418391,418391,418391
4,291647,393042,393042,393042,393042
...,...,...,...,...,...
1069,242623,245397,308927,374022,400132
1070,273737,264566,310614,366409,399463
1071,176087,259574,311786,349263,398573
1072,166628,234940,300933,337270,398235


#### `Value`

In [17]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Value {window}MA'] = stock_df['Value'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Value.*")

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,2.505503e+08,250550277,250550277,250550277,250550277
1,7.412127e+07,162335773,162335773,162335773,162335773
2,1.395790e+08,154750197,154750197,154750197,154750197
3,8.402544e+07,137069008,137069008,137069008,137069008
4,9.875900e+07,129407007,129407007,129407007,129407007
...,...,...,...,...,...
1069,3.042545e+08,312464732,400465360,471740056,522674708
1070,3.422828e+08,335049868,402237020,461881902,521894382
1071,2.234450e+08,327521951,403488419,441559163,520871719
1072,2.119214e+08,295706268,389075792,427727194,520484355


#### `#Trades`

In [18]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'#Trades {window}MA'] = stock_df['#Trades'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "#Trades.*")

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,23768,23768,23768,23768,23768
1,6804,15286,15286,15286,15286
2,18600,16390,16390,16390,16390
3,11824,15249,15249,15249,15249
4,8584,13916,13916,13916,13916
...,...,...,...,...,...
1069,21720,27432,29375,33517,32273
1070,30658,28645,29881,33189,32381
1071,24482,28920,30324,31881,32400
1072,28170,27736,30009,31267,32597


### Target columns

In [19]:
for window in cnst.TARGET_WINDOWS:
    stock_df[f'Target {window}D'] = stock_df['Close'].shift(-window)

stock_df.filter(regex = "Target.*")

,Target 3D,Target 7D,Target 15D,Target 30D
0,337.65,343.30,355.30,353.55
1,339.65,341.90,354.55,352.10
2,345.45,341.85,356.65,348.05
3,346.35,344.25,354.45,345.85
4,343.30,349.20,360.00,340.10
...,...,...,...,...
1069,1266.55,NaN,NaN,NaN
1070,1275.50,NaN,NaN,NaN
1071,NaN,NaN,NaN,NaN
1072,NaN,NaN,NaN,NaN


### Saving processed data

In [20]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-processed.parquet'), 
    index = False
)
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,2020-01-01,309.60,331.80,306.30,326.30,326.00,323.24,331.80,182.8,775112,2.505503e+08,23768,25.50,1,0,1,1,1,1,2020,2,1,1,1,326.00,326.00,326.00,326.00,25.50,25.50,25.50,25.50,323.24,323.24,323.24,323.24,775112,775112,775112,775112,250550277,250550277,250550277,250550277,23768,23768,23768,23768,337.65,343.30,355.30,353.55
1,2020-01-02,329.00,333.00,321.30,331.50,329.75,327.19,333.00,182.8,226542,7.412127e+07,6804,11.70,1,0,1,2,2,1,2020,3,2,1,1,327.88,327.88,327.88,327.88,18.60,18.60,18.60,18.60,325.22,325.22,325.22,325.22,500827,500827,500827,500827,162335773,162335773,162335773,162335773,15286,15286,15286,15286,339.65,341.90,354.55,352.10
2,2020-01-03,333.00,340.25,325.25,329.00,328.35,333.78,340.25,182.8,418178,1.395790e+08,18600,15.00,0,0,1,1,3,1,2020,4,3,1,1,328.03,328.03,328.03,328.03,17.40,17.40,17.40,17.40,328.07,328.07,328.07,328.07,473277,473277,473277,473277,154750197,154750197,154750197,154750197,16390,16390,16390,16390,345.45,341.85,356.65,348.05
3,2020-01-06,325.00,342.00,320.05,339.00,337.65,331.16,342.00,182.8,253734,8.402544e+07,11824,21.95,1,0,1,1,6,1,2020,0,6,1,3,330.44,330.44,330.44,330.44,18.54,18.54,18.54,18.54,328.84,328.84,328.84,328.84,418391,418391,418391,418391,137069008,137069008,137069008,137069008,15249,15249,15249,15249,346.35,344.25,354.45,345.85
4,2020-01-07,343.00,345.80,331.25,336.30,339.65,338.63,345.80,182.8,291647,9.875900e+07,8584,14.55,1,0,1,2,7,1,2020,1,7,1,1,332.28,332.28,332.28,332.28,17.74,17.74,17.74,17.74,330.80,330.80,330.80,330.80,393042,393042,393042,393042,129407007,129407007,129407007,129407007,13916,13916,13916,13916,343.30,349.20,360.00,340.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,2024-04-22,1244.30,1262.70,1242.50,1250.40,1255.55,1254.02,1444.90,748.9,242623,3.042545e+08,21720,20.20,1,0,0,2,22,4,2024,0,113,2,3,1269.23,1284.41,1258.19,1299.32,40.08,40.42,45.32,46.36,1276.13,1289.47,1261.72,1301.57,245397,308927,374022,400132,312464732,400465360,471740056,522674708,27432,29375,33517,32273,1266.55,NaN,NaN,NaN
1070,2024-04-23,1255.55,1261.80,1242.20,1245.25,1251.30,1250.41,1444.90,748.9,273737,3.422828e+08,30658,19.60,0,0,0,1,23,4,2024,1,114,2,1,1260.41,1282.24,1258.35,1299.66,39.88,38.53,43.46,45.86,1266.21,1288.10,1261.05,1301.75,264566,310614,366409,399463,335049868,402237020,461881902,521894382,28645,29881,33189,32381,1275.50,NaN,NaN,NaN
1071,2024-04-24,1261.95,1281.00,1256.40,1264.55,1262.60,1268.95,1444.90,748.9,176087,2.234450e+08,24482,24.60,1,0,0,1,24,4,2024,2,115,2,1,1256.72,1280.00,1260.28,1300.03,35.44,37.51,42.17,45.67,1260.56,1286.62,1263.12,1302.21,259574,311786,349263,398573,327521951,403488419,441559163,520871719,28920,30324,31881,32400,NaN,NaN,NaN,NaN
1072,2024-04-25,1273.10,1285.85,1257.05,1260.30,1266.55,1271.82,1444.90,748.9,166628,2.119214e+08,28170,28.80,1,0,0,2,25,4,2024,3,116,2,1,1256.35,1278.27,1263.18,1300.11,32.85,37.45,41.77,45.68,1259.13,1284.77,1265.82,1302.44,234940,300933,337270,398235,295706268,389075792,427727194,520484355,27736,30009,31267,32597,NaN,NaN,NaN,NaN


## Data standardization

### Stock price based columns

In [21]:
stock_price_cols = [
    'Open', 'High', 'Low', 'LTP', '52W H', '52W L'
] + (
    stock_df.filter(regex = "Close.*").columns.to_list() +
    stock_df.filter(regex = "Range.*").columns.to_list() +
    stock_df.filter(regex = "VWAP.*").columns.to_list() +
    stock_df.filter(regex = "Target.*").columns.to_list()
)

stock_price_cols

['Open',
 'High',
 'Low',
 'LTP',
 '52W H',
 '52W L',
 'Close',
 'Close 7MA',
 'Close 15MA',
 'Close 30MA',
 'Close 60MA',
 'Range',
 'Range 7MA',
 'Range 15MA',
 'Range 30MA',
 'Range 60MA',
 'VWAP',
 'VWAP 7MA',
 'VWAP 15MA',
 'VWAP 30MA',
 'VWAP 60MA',
 'Target 3D',
 'Target 7D',
 'Target 15D',
 'Target 30D']

In [22]:
stock_df[stock_price_cols] = stock_df[stock_price_cols].div(stock_df['Close'], axis = 0).round(3)
stock_df[stock_price_cols]

,Open,High,Low,LTP,52W H,52W L,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.950,1.018,0.940,1.001,1.018,0.561,1.0,1.000,1.000,1.000,1.000,0.078,0.078,0.078,0.078,0.078,0.992,0.992,0.992,0.992,0.992,1.036,1.053,1.090,1.085
1,0.998,1.010,0.974,1.005,1.010,0.554,1.0,0.994,0.994,0.994,0.994,0.035,0.056,0.056,0.056,0.056,0.992,0.986,0.986,0.986,0.986,1.030,1.037,1.075,1.068
2,1.014,1.036,0.991,1.002,1.036,0.557,1.0,0.999,0.999,0.999,0.999,0.046,0.053,0.053,0.053,0.053,1.017,0.999,0.999,0.999,0.999,1.052,1.041,1.086,1.060
3,0.963,1.013,0.948,1.004,1.013,0.541,1.0,0.979,0.979,0.979,0.979,0.065,0.055,0.055,0.055,0.055,0.981,0.974,0.974,0.974,0.974,1.026,1.020,1.050,1.024
4,1.010,1.018,0.975,0.990,1.018,0.538,1.0,0.978,0.978,0.978,0.978,0.043,0.052,0.052,0.052,0.052,0.997,0.974,0.974,0.974,0.974,1.011,1.028,1.060,1.001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,0.991,1.006,0.990,0.996,1.151,0.596,1.0,1.011,1.023,1.002,1.035,0.016,0.032,0.032,0.036,0.037,0.999,1.016,1.027,1.005,1.037,1.009,NaN,NaN,NaN
1070,1.003,1.008,0.993,0.995,1.155,0.598,1.0,1.007,1.025,1.006,1.039,0.016,0.032,0.031,0.035,0.037,0.999,1.012,1.029,1.008,1.040,1.019,NaN,NaN,NaN
1071,0.999,1.015,0.995,1.002,1.144,0.593,1.0,0.995,1.014,0.998,1.030,0.019,0.028,0.030,0.033,0.036,1.005,0.998,1.019,1.000,1.031,NaN,NaN,NaN,NaN
1072,1.005,1.015,0.992,0.995,1.141,0.591,1.0,0.992,1.009,0.997,1.026,0.023,0.026,0.030,0.033,0.036,1.004,0.994,1.014,0.999,1.028,NaN,NaN,NaN,NaN


### Volume based columns

In [23]:
volume_cols = stock_df.filter(regex = "Volume.*").columns.to_list()

volume_cols

['Volume', 'Volume 7MA', 'Volume 15MA', 'Volume 30MA', 'Volume 60MA']

In [24]:
stock_df[volume_cols] = stock_df[volume_cols].div(stock_df['Volume'], axis = 0).round(3)
stock_df[volume_cols]

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,2.211,2.211,2.211,2.211
2,1.0,1.132,1.132,1.132,1.132
3,1.0,1.649,1.649,1.649,1.649
4,1.0,1.348,1.348,1.348,1.348
...,...,...,...,...,...
1069,1.0,1.011,1.273,1.542,1.649
1070,1.0,0.966,1.135,1.339,1.459
1071,1.0,1.474,1.771,1.983,2.264
1072,1.0,1.410,1.806,2.024,2.390


### Value based columns

In [25]:
value_cols = stock_df.filter(regex = "Value.*").columns.to_list()

value_cols

['Value', 'Value 7MA', 'Value 15MA', 'Value 30MA', 'Value 60MA']

In [26]:
stock_df[value_cols] = stock_df[value_cols].div(stock_df['Value'], axis = 0).round(3)
stock_df[value_cols]

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,2.190,2.190,2.190,2.190
2,1.0,1.109,1.109,1.109,1.109
3,1.0,1.631,1.631,1.631,1.631
4,1.0,1.310,1.310,1.310,1.310
...,...,...,...,...,...
1069,1.0,1.027,1.316,1.550,1.718
1070,1.0,0.979,1.175,1.349,1.525
1071,1.0,1.466,1.806,1.976,2.331
1072,1.0,1.395,1.836,2.018,2.456


### Trade count based columns

In [27]:
trade_count_cols = stock_df.filter(regex = "#Trades.*").columns.to_list()

trade_count_cols

['#Trades', '#Trades 7MA', '#Trades 15MA', '#Trades 30MA', '#Trades 60MA']

In [28]:
stock_df[trade_count_cols] = stock_df[trade_count_cols].div(stock_df['#Trades'], axis = 0).round(3)
stock_df[trade_count_cols]

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,2.247,2.247,2.247,2.247
2,1.0,0.881,0.881,0.881,0.881
3,1.0,1.290,1.290,1.290,1.290
4,1.0,1.621,1.621,1.621,1.621
...,...,...,...,...,...
1069,1.0,1.263,1.352,1.543,1.486
1070,1.0,0.934,0.975,1.083,1.056
1071,1.0,1.181,1.239,1.302,1.323
1072,1.0,0.985,1.065,1.110,1.157


### Dropping unnecessary columns

In [29]:
stock_df = stock_df.drop(columns = ['Date', 'Close', 'Range', 'Volume', 'Value', '#Trades'])
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.950,1.018,0.940,1.001,0.992,1.018,0.561,1,0,1,1,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.078,0.078,0.078,0.078,0.992,0.992,0.992,0.992,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.036,1.053,1.090,1.085
1,0.998,1.010,0.974,1.005,0.992,1.010,0.554,1,0,1,2,2,1,2020,3,2,1,1,0.994,0.994,0.994,0.994,0.056,0.056,0.056,0.056,0.986,0.986,0.986,0.986,2.211,2.211,2.211,2.211,2.190,2.190,2.190,2.190,2.247,2.247,2.247,2.247,1.030,1.037,1.075,1.068
2,1.014,1.036,0.991,1.002,1.017,1.036,0.557,0,0,1,1,3,1,2020,4,3,1,1,0.999,0.999,0.999,0.999,0.053,0.053,0.053,0.053,0.999,0.999,0.999,0.999,1.132,1.132,1.132,1.132,1.109,1.109,1.109,1.109,0.881,0.881,0.881,0.881,1.052,1.041,1.086,1.060
3,0.963,1.013,0.948,1.004,0.981,1.013,0.541,1,0,1,1,6,1,2020,0,6,1,3,0.979,0.979,0.979,0.979,0.055,0.055,0.055,0.055,0.974,0.974,0.974,0.974,1.649,1.649,1.649,1.649,1.631,1.631,1.631,1.631,1.290,1.290,1.290,1.290,1.026,1.020,1.050,1.024
4,1.010,1.018,0.975,0.990,0.997,1.018,0.538,1,0,1,2,7,1,2020,1,7,1,1,0.978,0.978,0.978,0.978,0.052,0.052,0.052,0.052,0.974,0.974,0.974,0.974,1.348,1.348,1.348,1.348,1.310,1.310,1.310,1.310,1.621,1.621,1.621,1.621,1.011,1.028,1.060,1.001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,0.991,1.006,0.990,0.996,0.999,1.151,0.596,1,0,0,2,22,4,2024,0,113,2,3,1.011,1.023,1.002,1.035,0.032,0.032,0.036,0.037,1.016,1.027,1.005,1.037,1.011,1.273,1.542,1.649,1.027,1.316,1.550,1.718,1.263,1.352,1.543,1.486,1.009,NaN,NaN,NaN
1070,1.003,1.008,0.993,0.995,0.999,1.155,0.598,0,0,0,1,23,4,2024,1,114,2,1,1.007,1.025,1.006,1.039,0.032,0.031,0.035,0.037,1.012,1.029,1.008,1.040,0.966,1.135,1.339,1.459,0.979,1.175,1.349,1.525,0.934,0.975,1.083,1.056,1.019,NaN,NaN,NaN
1071,0.999,1.015,0.995,1.002,1.005,1.144,0.593,1,0,0,1,24,4,2024,2,115,2,1,0.995,1.014,0.998,1.030,0.028,0.030,0.033,0.036,0.998,1.019,1.000,1.031,1.474,1.771,1.983,2.264,1.466,1.806,1.976,2.331,1.181,1.239,1.302,1.323,NaN,NaN,NaN,NaN
1072,1.005,1.015,0.992,0.995,1.004,1.141,0.591,1,0,0,2,25,4,2024,3,116,2,1,0.992,1.009,0.997,1.026,0.026,0.030,0.033,0.036,0.994,1.014,0.999,1.028,1.410,1.806,2.024,2.390,1.395,1.836,2.018,2.456,0.985,1.065,1.110,1.157,NaN,NaN,NaN,NaN


### Saving standardized data

In [30]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-standardized.parquet'), 
    index = False
)
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.950,1.018,0.940,1.001,0.992,1.018,0.561,1,0,1,1,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.078,0.078,0.078,0.078,0.992,0.992,0.992,0.992,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.036,1.053,1.090,1.085
1,0.998,1.010,0.974,1.005,0.992,1.010,0.554,1,0,1,2,2,1,2020,3,2,1,1,0.994,0.994,0.994,0.994,0.056,0.056,0.056,0.056,0.986,0.986,0.986,0.986,2.211,2.211,2.211,2.211,2.190,2.190,2.190,2.190,2.247,2.247,2.247,2.247,1.030,1.037,1.075,1.068
2,1.014,1.036,0.991,1.002,1.017,1.036,0.557,0,0,1,1,3,1,2020,4,3,1,1,0.999,0.999,0.999,0.999,0.053,0.053,0.053,0.053,0.999,0.999,0.999,0.999,1.132,1.132,1.132,1.132,1.109,1.109,1.109,1.109,0.881,0.881,0.881,0.881,1.052,1.041,1.086,1.060
3,0.963,1.013,0.948,1.004,0.981,1.013,0.541,1,0,1,1,6,1,2020,0,6,1,3,0.979,0.979,0.979,0.979,0.055,0.055,0.055,0.055,0.974,0.974,0.974,0.974,1.649,1.649,1.649,1.649,1.631,1.631,1.631,1.631,1.290,1.290,1.290,1.290,1.026,1.020,1.050,1.024
4,1.010,1.018,0.975,0.990,0.997,1.018,0.538,1,0,1,2,7,1,2020,1,7,1,1,0.978,0.978,0.978,0.978,0.052,0.052,0.052,0.052,0.974,0.974,0.974,0.974,1.348,1.348,1.348,1.348,1.310,1.310,1.310,1.310,1.621,1.621,1.621,1.621,1.011,1.028,1.060,1.001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,0.991,1.006,0.990,0.996,0.999,1.151,0.596,1,0,0,2,22,4,2024,0,113,2,3,1.011,1.023,1.002,1.035,0.032,0.032,0.036,0.037,1.016,1.027,1.005,1.037,1.011,1.273,1.542,1.649,1.027,1.316,1.550,1.718,1.263,1.352,1.543,1.486,1.009,NaN,NaN,NaN
1070,1.003,1.008,0.993,0.995,0.999,1.155,0.598,0,0,0,1,23,4,2024,1,114,2,1,1.007,1.025,1.006,1.039,0.032,0.031,0.035,0.037,1.012,1.029,1.008,1.040,0.966,1.135,1.339,1.459,0.979,1.175,1.349,1.525,0.934,0.975,1.083,1.056,1.019,NaN,NaN,NaN
1071,0.999,1.015,0.995,1.002,1.005,1.144,0.593,1,0,0,1,24,4,2024,2,115,2,1,0.995,1.014,0.998,1.030,0.028,0.030,0.033,0.036,0.998,1.019,1.000,1.031,1.474,1.771,1.983,2.264,1.466,1.806,1.976,2.331,1.181,1.239,1.302,1.323,NaN,NaN,NaN,NaN
1072,1.005,1.015,0.992,0.995,1.004,1.141,0.591,1,0,0,2,25,4,2024,3,116,2,1,0.992,1.009,0.997,1.026,0.026,0.030,0.033,0.036,0.994,1.014,0.999,1.028,1.410,1.806,2.024,2.390,1.395,1.836,2.018,2.456,0.985,1.065,1.110,1.157,NaN,NaN,NaN,NaN
